In [11]:
class Vec2D:
    def __init__(self, x: float, y: float) -> None:
        self.x = x
        self.y = y

    @staticmethod
    def zero():
        return Vec3D(0, 0, 0)

    def __add__(self, other):
        return Vec3D(self.x + other.x, self.y + other.y)

    def __iter__(self):
        yield self.x
        yield self.y

    def __repr__(self) -> str:
        return f"Vec2D({self.x}, {self.y})"


class Vec3D:
    @staticmethod
    def zero():
        return Vec3D(0, 0, 0)

    def __add__(self, other):
        return Vec3D(self.x + other.x, self.y + other.y, self.z + other.z)

    def __sub__(self, other):
        return self + -other

    def __neg__(self):
        return Vec3D(-self.x, -self.y, -self.z)

    def __repr__(self) -> str:
        return f"Vec3D({self.x}, {self.y}, {self.z})"

    def __init__(self, x: float, y: float, z: float) -> None:
        self.x = x
        self.y = y
        self.z = z

    def dot(self, other):
        return self.x * other.x + self.y * other.y + self.z * other.z

    def unit_dot(self, other):
        return self.unit().dot(other.unit())

    def unit(self):
        mag = self.magnitude()
        return Vec3D(self.x / mag, self.y / mag, self.z / mag)

    def magnitude(self):
        return (self.x**2 + self.y**2 + self.z**2)**0.5 

    def __iter__(self):
        yield self.x
        yield self.y
        yield self.z


class Rect2D:
    def __init__(self, pos: Vec2D, size: Vec2D) -> None:
        self.position = pos
        self.size = size

    @staticmethod
    def from_center_size(pos_center: Vec2D, size: Vec2D):
        position = Vec2D(pos_center.x - size.x / 2, pos_center.y - size.y / 2)
        return Rect2D(position, size)

    def left(self):
        return self.position.x

    def right(self):
        return self.position.x + self.size.x

    def top(self):
        return self.position.y

    def bottom(self):
        return self.position.y + self.size.y

    def width(self):
        return self.size.x

    def height(self):
        return self.size.y


class View:
    def __init__(
        self, position: Vec2D, canvas_rect: Rect2D, image_rect: Rect2D, near_clip: float
    ) -> None:
        self.position = position
        self.canvas_rect = canvas_rect
        self.image_rect = image_rect
        self.near_clip = near_clip


class Triangle:
    def __init__(self, color: str, *vertices) -> None:
        self.color = color
        self.verticies = list(vertices)
        self.albedo = 0.18

    def __iter__(self):
        for vertex in self.verticies:
            yield vertex



class Shape3D:
    def __init__(self, *args) -> None:
        self.triangles = list(args)

    def __iter__(self):
        for triangle in self.triangles:
            yield triangle

class Light:
    def __init__(self, position: Vec3D, color: str, intensity: float):
        self.position = position
        self.color = color
        self.intensity = intensity

    def reflect(triangle: Triangle):
        


import math
import drawSvg as draw

# cam_po = point in camera space
def project(view: View, cam_pt: Vec3D) -> Vec2D:
    # print(view.near_clip, cam_pt.x, cam_pt.z)
    screen_x = view.near_clip * cam_pt.x / -cam_pt.z
    screen_y = view.near_clip * cam_pt.y / -cam_pt.z

    screen_pt = Vec2D(screen_x, screen_y)

    # if (abs(screen_x) >= view.canvas_rect.width() / 2) or (
    #     abs(screen_y) >= view.canvas_rect.height() / 2
    # ):
    #     print(f"screen pt{screen_pt} is out of screen bounds")
    #     return None

    print(f"screen pt: {screen_pt}")
    return screen_pt


def map_to_ndc(view: View, screen_pt: Vec3D) -> Vec2D:
    # ndc_pt_x = (screen_pt.x + (view.canvas_rect.width() / 2)) / view.canvas_rect.width()
    # ndc_pt_y = (
    #     screen_pt.y + (view.canvas_rect.height() / 2)
    # ) / view.canvas_rect.height()
    ndc_pt_x = (screen_pt.x / view.canvas_rect.width()) + 0.5
    ndc_pt_y = (screen_pt.y / view.canvas_rect.height()) + 0.5
    ndc_pt = Vec2D(ndc_pt_x, ndc_pt_y)
    print(f"ndc pt: {ndc_pt}")
    return ndc_pt


def rasterize(view: View, ndc_pt: Vec2D, cam_pt: Vec3D) -> Vec3D:
    rast_x = math.floor(ndc_pt.x * view.image_rect.width())
    rast_y = math.floor((1 - ndc_pt.y) * view.image_rect.height())
    rast_z = -cam_pt.z

    rast_pt = Vec3D(rast_x, rast_y, rast_z)
    if rast_pt.z <= 0:
        print(f"rast pt {rast_pt} behind camera")
        return None

    print(f"rast pt: {rast_pt}\n")
    return rast_pt


def cam_pt_to_rast_pt(view: View, cam_pt: Vec2D):
    screen_pt = project(view, cam_pt)
    if screen_pt != None:
        ndc_pt = map_to_ndc(view, screen_pt)
        rast_pt = rasterize(view, ndc_pt, cam_pt)
        return rast_pt


def generate_cube(center_pos: Vec3D, size: float) -> Shape3D:
    d = size / 2  # delta
    left_top_front = center_pos + Vec3D(-d, d, d)
    left_top_back = center_pos + Vec3D(-d, d, -d)
    left_bottom_front = center_pos + Vec3D(-d, -d, d)
    left_bottom_back = center_pos + Vec3D(-d, -d, -d)

    right_top_front = center_pos + Vec3D(d, d, d)
    right_top_back = center_pos + Vec3D(d, d, -d)
    right_bottom_front = center_pos + Vec3D(d, -d, d)
    right_bottom_back = center_pos + Vec3D(d, -d, -d)

    left_lower = Triangle(left_top_back, left_bottom_back, left_bottom_front)
    left_upper = Triangle(left_top_back, left_top_front, left_bottom_front)

    right_lower = Triangle(right_top_back, right_bottom_back, right_bottom_front)
    right_upper = Triangle(right_top_back, right_top_front, right_bottom_front)

    front_lower = Triangle(left_bottom_front, left_top_front, right_bottom_front)
    front_upper = Triangle(right_top_front, left_top_front, right_bottom_front)

    back_lower = Triangle(left_bottom_back, left_top_back, right_bottom_back)
    back_upper = Triangle(right_top_back, left_top_back, right_bottom_back)

    top_lower = Triangle(left_top_front, left_top_back, right_top_front)
    top_upper = Triangle(right_top_back, left_top_back, right_top_front)

    bottom_lower = Triangle(left_bottom_front, left_bottom_back, right_bottom_front)
    bottom_upper = Triangle(right_bottom_back, left_bottom_back, right_bottom_front)

    return Shape3D(
        left_lower,
        left_upper,
        right_lower,
        right_upper,
        front_lower,
        front_upper,
        back_lower,
        back_upper,
        top_lower,
        top_upper,
        bottom_lower,
        bottom_upper,
    )

    pass

def triangle_normal(triangle: Triangle) -> Vec3D:
    u = triangle.verticies[1] - triangle.verticies[0] 
    v = triangle.verticies[2] - triangle.verticies[0] 
    print(u, v)

    n_x = (u.y * v.z) - (u.z * v.y)
    n_y = (u.z * v.x) - (u.x * v.z)
    n_z = (u.x * v.y) - (u.y * v.x)

    return Vec3D(n_x, n_y, n_z)

def render_point3d_to_svg(view: View, svg, point3d):
    if point3d != None:
        point2d = cam_pt_to_rast_pt(view, point3d)
        print(point2d)
        if point2d != None:
            svg.append(
                draw.Circle(
                    point2d.x, (view.image_rect.height() - point2d.y), 2, fill="white"
                )
            )


def render_triangle2d(svg, triangle2d):
    svg.append(
        draw.Lines(
            *triangle2d,
            close=False,
            fill=triangle2d.color,
            stroke="black",
        )
    )


def render_triangle_3d_to_2d(view: View, svg, triangle3d):
    points3d = list(triangle3d)
    points2d = [
        pt_coord
        for pt in map(
            lambda pt: (pt.x, pt.y),
            filter(
                lambda pt: pt != None,
                map(lambda pt: cam_pt_to_rast_pt(view, pt), points3d),
            ),
        )
        for pt_coord in pt
    ]
    triangle2d = Triangle(triangle3d.color, *points2d)
    render_triangle2d(svg, triangle2d)


def render_shape3d_to_svg(view: View, svg, shape: Shape3D):
    for triangle in shape:
        render_triangle_3d_to_2d(view, svg, triangle)


def main():
    view_pos = Vec3D.zero()
    view_canvas_rect = Rect2D.from_center_size(Vec2D.zero(), Vec2D(10, 10))
    view_image_rect = Rect2D(Vec2D.zero(), Vec2D(512, 384))

    view = View(view_pos, view_canvas_rect, view_image_rect, 1)
    point_3d = Vec3D(20, 23, -2)
    # point_2d = cam_pt_to_rast_pt(view, point_3d)

    t = Triangle(
        '#f0f',
        Vec3D(-1, -1, -2),
        Vec3D(1, -1.5, -2),
        Vec3D(1, 1, -2)
    )

    t2 = Triangle('#f0f',
        Vec3D(-3, -3, -4),
        Vec3D(-4, 3, -2),
        Vec3D(1, 4, -4)
    )

    svg = draw.Drawing(
        view.image_rect.width(),
        view.image_rect.height(),
        origin=(0, 0),
        displayInline=False,
    )

    bg = draw.Rectangle(
        view_image_rect.position.x,
        view_image_rect.position.y,
        view_image_rect.width(),
        view_image_rect.height(),
        fill="#000000",
    )
    svg.append(bg)

    cube = generate_cube(Vec3D(90, 150, -300), 200)

    light = Light(Vec3D(0, 0, 0), '#fff', 1)

    render_triangle_3d_to_2d(view, svg, t)
    render_triangle_3d_to_2d(view, svg, t2)

    print('---------------------------')
    print('normal:', triangle_normal(t))

    svg.rasterize()
    return svg


# run in a jupyter notebook
main()


screen pt: Vec2D(-0.5, -0.5)
ndc pt: Vec2D(0.45, 0.45)
rast pt: Vec3D(230, 211, 2)

screen pt: Vec2D(0.5, -0.75)
ndc pt: Vec2D(0.55, 0.425)
rast pt: Vec3D(281, 220, 2)

screen pt: Vec2D(0.5, 0.5)
ndc pt: Vec2D(0.55, 0.55)
rast pt: Vec3D(281, 172, 2)

screen pt: Vec2D(-0.75, -0.75)
ndc pt: Vec2D(0.425, 0.425)
rast pt: Vec3D(217, 220, 4)

screen pt: Vec2D(-2.0, 1.5)
ndc pt: Vec2D(0.3, 0.65)
rast pt: Vec3D(153, 134, 2)

screen pt: Vec2D(0.25, 1.0)
ndc pt: Vec2D(0.525, 0.6)
rast pt: Vec3D(268, 153, 4)

---------------------------
Vec3D(2, -0.5, 0) Vec3D(2, 2, 0)
normal: Vec3D(-0.0, 0, 5.0)


In [17]:
Vec3D(1, 0, 0).unit_dot(Vec3D(0, 0, -1))

0.0